<a href="https://colab.research.google.com/github/rajy4683/MonoMaskDepth/blob/master/DataGeneratorPipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sun May 10 16:36:01 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import sys

Mounted at /content/drive


In [0]:
!pwd

/content


In [0]:
!folder_list=("classroom" "college_outdoors" "corridors" "dining_room" "malls" "lobby" "meeting_rooms" "people_cropresize"); for folder_name in ${folder_list[@]}; do mkdir -p /content/S15EVA4/$folder_name /content/S15EVA4/$folder_name/aug_images; cp -Rf /content/drive/My\ Drive/EVA4/tsai/S15EVA4/$folder_name/*png /content/S15EVA4/$folder_name; done

In [0]:
sys.path.append('/content/drive/My Drive/EVA4/RekogNizer')
sys.path.append('/content/drive/My Drive/EVA4')
sys.path.append('/content/drive/My Drive/EVA4/DenseDepth')

### Dependencies


In [3]:
!pip install -r /content/drive/My\ Drive/EVA4/RekogNizer/requirements.txt

     |████████████████████████████████| 112kB 46.1MB/s 
     |████████████████████████████████| 1.4MB 44.9MB/s 
     |████████████████████████████████| 112kB 51.9MB/s 
     |████████████████████████████████| 102kB 15.0MB/s 
     |████████████████████████████████| 102kB 14.7MB/s 
     |████████████████████████████████| 460kB 53.3MB/s 
     |████████████████████████████████| 634kB 53.2MB/s 
     |████████████████████████████████| 71kB 9.9MB/s 
     |████████████████████████████████| 71kB 11.3MB/s 
  Created wheel for gql: filename=gql-0.2.0-cp36-none-any.whl size=7630 sha256=a9decda73f40e2de1b29642aaaa3a564f8208d1386b50fdd39fa6154563c141f
  Stored in directory: /root/.cache/pip/wheels/ce/0e/7b/58a8a5268655b3ad74feef5aa97946f0addafb3cbb6bd2da23
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=eb7e7b1c21e653537d76450aa7663813d89f335f5d9bc6660318c2cce6ecde90
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f

In [4]:
!pip install -U git+https://github.com/albu/albumentations

  Cloning https://github.com/albu/albumentations to /tmp/pip-req-build-i8x4tm2p
  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-i8x4tm2p
  Created wheel for albumentations: filename=albumentations-0.4.5-cp36-none-any.whl size=65055 sha256=98440d8183354ea0039a55d30b9280293eaecce15d1fea7441eb93e7a8ca4faa
  Stored in directory: /tmp/pip-ephem-wheel-cache-l63gemot/wheels/45/8b/e4/2837bbcf517d00732b8e394f8646f22b8723ac00993230188b
Successfully built albumentations
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [7]:
import os

import numpy as np
import cv2
from matplotlib import pyplot as plt
from skimage.color import label2rgb

import albumentations as A
import random
from google.colab.patches import cv2_imshow
from PIL import Image
import glob
from zipfile import ZipFile
from RekogNizer.depthmodelutils import * ### Contains all the utils developed for transforms handling.

/content/S15EVA4/classroom/*png
/content/S15EVA4/college_outdoors/*png
/content/S15EVA4/corridors/*png
/content/S15EVA4/dining_room/*png
/content/S15EVA4/lobby/*png
/content/S15EVA4/malls/*png
/content/S15EVA4/meeting_rooms/*png
/content/S15EVA4/people_cropresize/*png


In [0]:
def load_return_fgbg_lists():
    base_dir = "/content/S15EVA4/"
    file_format = "*png"
    folder_list = ["classroom", "college_outdoors", 
                   "corridors", "dining_room",
                   "lobby","malls", "meeting_rooms",
                   "people_cropresize"]
    folder_dict = dict()
    for folder_name in folder_list:
        full_path = base_dir+folder_name+"/"+"*png"
        print(full_path)
        folder_dict[folder_name] = [ val for val in glob.glob(full_path)]
    return folder_dict


In [0]:
### Dict of all the filenames indexed by background image type
fg_bg_file_list = load_return_fgbg_lists()

/content/S15EVA4/classroom/*png
/content/S15EVA4/college_outdoors/*png
/content/S15EVA4/corridors/*png
/content/S15EVA4/dining_room/*png
/content/S15EVA4/lobby/*png
/content/S15EVA4/malls/*png
/content/S15EVA4/meeting_rooms/*png
/content/S15EVA4/people_cropresize/*png


### Foreground with masks, flips and masks of flips

In [0]:
%cd /content/S15EVA4/

In [0]:
generate_fg_set(fg_bg_file_list['people_cropresize'])#[1]

In [0]:
fg_bg_file_list.keys()

dict_keys(['classroom', 'college_outdoors', 'corridors', 'dining_room', 'lobby', 'malls', 'meeting_rooms', 'people_cropresize'])

In [0]:
large_bg_file_list = []
for iter in list(fg_bg_file_list.keys())[:-1]:
    large_bg_file_list.extend(fg_bg_file_list[iter])

In [0]:
large_file_list = [ (bg, fg) for bg in large_bg_file_list for fg in fg_bg_file_list['people_cropresize']]

In [0]:
len(large_bg_file_list),len(large_file_list)

(0, 0)

### Triggers the mammoth overlay operations

In [0]:
from tqdm import tqdm
import os
import pandas as pd

df_file_list = pd.read_csv("/content/drive/My Drive/EVA4/tsai/S15EVA4/fg_bg.csv")
pbar = tqdm(df_file_list.loc[8000:,["Background","Foreground"]].values)
counter = 0
file_count = 0
for background_file, foreground_file in pbar:
    counter=counter+1
    if(counter%100 == 0 ):
        file_count = len([ val for folder_name in list(fg_bg_file_list.keys())[:-1] for val in glob.glob("/content/S15EVA4/"+folder_name+"/aug_images/*jpg")])
        #print("Total files:{}".format(file_count))
    status_desc = "Currently handling: {}:{} {} File Count:{}".format(counter,os.path.basename(background_file), os.path.basename(foreground_file), file_count)
    pbar.set_description(status_desc)    
    overlay_images(background_file, foreground_file,replica_count=20, save_images=True, debug=False)

Currently handling: 4100:meetin_room_9.png people_188.png File Count:327920: 100%|██████████| 4100/4100 [18:17<00:00,  3.74it/s]


In [0]:
file_list = [ val for folder_name in list(fg_bg_file_list.keys())[:-1] for val in glob.glob("/content/S15EVA4/"+folder_name+"/aug_images/*jpg")]
print("Total files to be zipped:{}".format(len(file_list)))
from zipfile import ZipFile 
import os 
with ZipFile('my_python_files_8k12k.zip','w') as zip: 
    # writing each file one by one 
    for file in file_list: 
        zip.write(file)

Total files to be zipped:328000


In [0]:
!mv /content/S15EVA4/my_python_files_8k12k.zip /content/drive/My\ Drive/EVA4/tsai/S15EVA4

### Depth Model invocation to create Depth Images

In [0]:
%cd /content/drive/My\ Drive/EVA4

/content/drive/My Drive/EVA4


In [0]:
!git clone https://github.com/ialhashim/DenseDepth.git

Cloning into 'DenseDepth'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 235 (delta 0), reused 1 (delta 0), pack-reused 231
Receiving objects: 100% (235/235), 11.80 MiB | 14.20 MiB/s, done.
Resolving deltas: 100% (113/113), done.


In [0]:
!wget https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5 -O /content/nyu.h5

--2020-05-03 10:46:05--  https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.65.4
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.65.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 172897376 (165M) [application/h5]
Saving to: ‘/content/nyu.h5’

/content/nyu.h5     100%[===================>] 164.89M  11.6MB/s    in 16s     

2020-05-03 10:46:22 (10.4 MB/s) - ‘/content/nyu.h5’ saved [172897376/172897376]



In [0]:
!python test.py --model "/content/nyu.h5" --input "/content/my_python_files_8k12k.zip" --chunk_size 300

In [0]:
file_list = [ folder_name for folder_name in glob.glob("/content/test/*jpg")]
print("Total files to be zipped:{}".format(len(file_list)))
from zipfile import ZipFile 
import os 
with ZipFile('/content/depth_8k12k.zip','w') as zip: 
    # writing each file one by one 
    for file in file_list: 
        zip.write(file,arcname=os.path.basename(file))

Total files to be zipped:163700
